In [179]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
# https://github.com/arpanghosh8453/programs/blob/master/Fitbit%20Data%20Analyzer/Fitbit%20HR%20analyzer.ipynb
# https://github.com/lkippenb/Machine-Learning-Fitbit-Sleep/blob/master/analysis.ipynb
# https://github.com/yashatgit/fitbit-analyzer/blob/master/Fitbit_Data_Analysis.ipynb
# https://github.com/yashatgit/fitbit-analyzer/blob/master/Fitbit_Data_Collection.ipynb

# https://dev.fitbit.com/
# https://dev.fitbit.com/build/reference/web-api/basics/

# You need the following modules, if you don't have them, use pip install <module-name>
import requests
import pandas as pd
import datetime
import numpy as np
import json

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import jupyter_dash
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Heartrate

In [2]:
hr_df = pd.read_csv('~/qs/fitbit/data/hr.csv')
hr_df

,time,value
0,2021-01-01 00:00:00,59
1,2021-01-01 00:01:00,62
2,2021-01-01 00:02:00,59
3,2021-01-01 00:03:00,60
4,2021-01-01 00:04:00,59
...,...,...
1381,2021-01-01 23:55:00,57
1382,2021-01-01 23:56:00,61
1383,2021-01-01 23:57:00,62
1384,2021-01-01 23:58:00,60


In [8]:
title = 'Fitbit HR Dashboard'

df = hr_df
mean_hr = round(df['value'].mean(), 1)
max_hr = df['value'].max()
min_hr = df['value'].min()

# rolling=10
# rolling_hr = df['value'].rolling(rolling).mean()

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['time'], y=df['value'], name='Heart Rate', connectgaps=False))
#fig.add_trace(go.Scatter(x=df['time'], y=rolling_hr, name=f'{rolling} Minute Rolling Heart Rate'))

# fig.add_shape(type="line", x0=min(df['time']), y0=mean_hr, x1=max(df['time']), y1=mean_hr, line_dash="dash")
fig.add_hline(y=mean_hr, line_dash="dash",
              #annotation_text=f"Mean HR: {mean_hr}", 
              #annotation_position="bottom right"
             )
# fig.add_vrect(x0="2018-09-24", x1="2018-12-18", 
#               annotation_text="decline", annotation_position="top left",
#               fillcolor="green", opacity=0.25, line_width=0)

fig.update_yaxes(title_text="Heart Rate")

fig.update_layout(title=dict(text=title, 
                             yanchor="top",
                             y=.95,
                             xanchor="left",
                             x=.05),
                 legend=dict(orientation="h",
                             yanchor="bottom",
                             y=1.02,
                             xanchor="right",
                             x=1))

fig.update_layout(
    hovermode='x unified',
    dragmode='pan',
    annotations=[
        #https://plotly.com/python/text-and-annotations/#adding-annotations-with-xref-and-yref-as-paper
        #https://community.plotly.com/t/how-to-add-a-text-area-with-custom-text-as-a-kind-of-legend-in-a-plotly-plot/24349/3
        go.layout.Annotation(
            text=f'Max HR: {max_hr}<br>Min HR: {min_hr}<br>Mean HR: {mean_hr}',
            align='left',
            showarrow=False,
            xref='paper',
            yref='paper',
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
          )],
    xaxis=dict(
        
        #change to set view window
        range=[min(df['time']), df['time']],
        
        rangeselector=dict(
            buttons=list([
                dict(count=6,
                     label="6hr",
                     step="hour",
                     stepmode="backward"),
                dict(count=1,
                     label="24hr",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="1wk",
                     step="day",
                     stepmode="backward"),
                dict(step="all"),
            ])
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '95vh'}
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=8040, debug=True)

Dash app running on http://127.0.0.1:8040/


In [4]:
# hull moving average?
# HMA[i] = MA( (2*MA(input, period/2) – MA(input, period)), SQRT(period)) where MA is a moving average and SQRT is square root.

In [5]:
df['value'].quantile(.25)

56.0

In [6]:
df['value'].quantile(.75)

69.0

# Step Count

In [7]:
step_df = pd.read_csv('~/qs/fitbit/data/steps.csv')
step_df

,time,Step Count
0,2020-01-01,1041
1,2020-01-02,321
2,2020-01-03,1659
3,2020-01-04,8407
4,2020-01-05,4520
...,...,...
361,2020-12-27,4143
362,2020-12-28,5340
363,2020-12-29,3503
364,2020-12-30,6311


In [64]:
title = 'Fitbit Steps Dashboard'

df = step_df
mean_steps = round(df['Step Count'].mean())
max_steps = df['Step Count'].max()
min_steps = df['Step Count'].min()

rolling=7
rolling_steps = round(df['Step Count'].rolling(rolling).mean())

fig = go.Figure()

fig.add_trace(go.Bar(x=df['time'], y=df['Step Count'], name='Step Count'))
fig.add_trace(go.Scatter(x=df['time'], y=rolling_steps, name=f'{rolling} Day Rolling Steps'))

fig.add_hline(y=mean_steps, line_dash="dash")

fig.update_yaxes(title_text="Step Count")
fig.update_xaxes(title_text="Date")
fig.update_layout(title=dict(text=title, 
                             yanchor="top",
                             y=.95,
                             xanchor="left",
                             x=.05),
                 legend=dict(orientation="h",
                             yanchor="bottom",
                             y=1.02,
                             xanchor="right",
                             x=1))

fig.update_layout(
    hovermode='x unified',
    dragmode='pan',
    annotations=[
        #https://plotly.com/python/text-and-annotations/#adding-annotations-with-xref-and-yref-as-paper
        #https://community.plotly.com/t/how-to-add-a-text-area-with-custom-text-as-a-kind-of-legend-in-a-plotly-plot/24349/3
        go.layout.Annotation(
            text=f'Max Steps: {max_steps}<br>Min Steps: {min_steps}<br>Mean Steps: {mean_steps}',
            align='left',
            showarrow=False,
            xref='paper',
            yref='paper',
            yanchor="top",
            y=.98,
            xanchor="right",
            x=1,
          )],
    xaxis=dict(
        
        # change to set view window
        # range=[min(df['time']), df['time'][60]],
        
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label="1wk",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1MO",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6MO",
                     step="month",
                     stepmode="backward"),
                dict(step="all"),
            ])
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '95vh'}
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=8080, debug=True)

Dash app running on http://127.0.0.1:8080/


/home/mcwaage1/venv/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



# Distance

In [22]:
distance_df = pd.read_csv('~/qs/fitbit/data/distance.csv')
distance_df

,time,Distance in KM,Distance in MI
0,2020-01-01,0.82576,0.513102
1,2020-01-02,0.24246,0.150657
2,2020-01-03,1.25603,0.780459
3,2020-01-04,6.37055,3.958469
4,2020-01-05,3.49040,2.168830
...,...,...,...
362,2020-12-28,4.04441,2.513075
363,2020-12-29,2.68446,1.668043
364,2020-12-30,4.77629,2.967843
365,2020-12-31,2.75618,1.712608


In [27]:
distance_df['Distance in KM'] = distance_df['Distance in KM'].apply(lambda x: round(x, 2))
distance_df['Distance in MI'] = distance_df['Distance in MI'].apply(lambda x: round(x, 2))
distance_df

,time,Distance in KM,Distance in MI
0,2020-01-01,0.83,0.51
1,2020-01-02,0.24,0.15
2,2020-01-03,1.26,0.78
3,2020-01-04,6.37,3.96
4,2020-01-05,3.49,2.17
...,...,...,...
362,2020-12-28,4.04,2.51
363,2020-12-29,2.68,1.67
364,2020-12-30,4.78,2.97
365,2020-12-31,2.76,1.71


In [29]:
title = 'Fitbit Distance Dashboard'

df = distance_df
mean_distance = round(df['Distance in MI'].mean(), 2)
max_distance = round(df['Distance in MI'].max(), 2)
min_distance = round(df['Distance in MI'].min(), 2)

rolling=7
rolling_distance = round(df['Distance in MI'].rolling(rolling).mean(), 2)

fig = go.Figure()

fig.add_trace(go.Bar(x=df['time'], y=df['Distance in MI'], name='Distance in MI'))
fig.add_trace(go.Scatter(x=df['time'], y=rolling_distance, name=f'{rolling} Day Rolling Distance'))

fig.add_hline(y=mean_distance, line_dash="dash")

fig.update_yaxes(title_text="Distance in MI")
fig.update_xaxes(title_text="Date")
fig.update_layout(title=dict(text=title, 
                             yanchor="top",
                             y=.95,
                             xanchor="left",
                             x=.05),
                 legend=dict(orientation="h",
                             yanchor="bottom",
                             y=1.02,
                             xanchor="right",
                             x=1))

fig.update_layout(
    hovermode='x unified',
    dragmode='pan',
    annotations=[
        #https://plotly.com/python/text-and-annotations/#adding-annotations-with-xref-and-yref-as-paper
        #https://community.plotly.com/t/how-to-add-a-text-area-with-custom-text-as-a-kind-of-legend-in-a-plotly-plot/24349/3
        go.layout.Annotation(
            text=f'Max Distance: {max_distance}<br>Min Distance: {min_distance}<br>Mean Distance: {mean_distance}',
            align='left',
            showarrow=False,
            xref='paper',
            yref='paper',
            yanchor="top",
            y=.98,
            xanchor="right",
            x=1,
          )],
    xaxis=dict(
        
        # change to set view window
        # range=[min(df['time']), df['time'][60]],
        
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label="1wk",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1MO",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6MO",
                     step="month",
                     stepmode="backward"),
                dict(step="all"),
            ])
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '95vh'}
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=8090, debug=True)

Dash app running on http://127.0.0.1:8090/


# Elevation

In [45]:
elevation_df = pd.read_csv('~/qs/fitbit/data/elevation.csv')
elevation_df

,Date,Elevation in Meters,Elevation in Floors
0,2020-01-01,0.0,0.0
1,2020-01-02,0.0,0.0
2,2020-01-03,3.0,1.0
3,2020-01-04,24.0,8.0
4,2020-01-05,6.0,2.0
...,...,...,...
362,2020-12-28,9.0,3.0
363,2020-12-29,3.0,1.0
364,2020-12-30,6.0,2.0
365,2020-12-31,3.0,1.0


In [62]:
# https://plotly.com/python/dropdowns/#relayout-dropdown
# https://dash.plotly.com/dash-core-components/dropdown
# add dropdown to update traces between meters and floors


df = elevation_df

title = 'Fitbit Elevation Dashboard'

mean_elevation_meters = round(df['Elevation in Meters'].mean(), 2)
max_elevation_meters = round(df['Elevation in Meters'].max(), 2)
min_elevation_meters = round(df['Elevation in Meters'].min(), 2)

rolling=7
rolling_elevation_meters = round(df['Elevation in Meters'].rolling(rolling).mean(), 2)

fig = go.Figure()

fig.add_trace(go.Bar(x=df['Date'], y=df['Elevation in Meters'], name='Elevation in Meters'))

fig.add_trace(go.Scatter(x=df['Date'], y=rolling_elevation_meters, name=f'{rolling} Day Rolling Elevation in Meters'))

fig.add_hline(y=mean_elevation_meters, line_dash="dash")

fig.update_yaxes(title_text="Elevation")
fig.update_xaxes(title_text="Date")
fig.update_layout(title=dict(text=title, 
                             yanchor="top",
                             y=.95,
                             xanchor="left",
                             x=.05),
                 legend=dict(orientation="h",
                             yanchor="bottom",
                             y=1.02,
                             xanchor="right",
                             x=1))

fig.update_layout(
    hovermode='x unified',
    dragmode='pan',
    annotations=[
        #https://plotly.com/python/text-and-annotations/#adding-annotations-with-xref-and-yref-as-paper
        #https://community.plotly.com/t/how-to-add-a-text-area-with-custom-text-as-a-kind-of-legend-in-a-plotly-plot/24349/3
        go.layout.Annotation(
            text=f'Max Elevation: {max_elevation}<br>Min Elevation: {min_elevation}<br>Mean Elevation: {mean_elevation}',
            align='left',
            showarrow=False,
            xref='paper',
            yref='paper',
            yanchor="top",
            y=.98,
            xanchor="right",
            x=1,
          )],
    xaxis=dict(
        
        # change to set view window
        # range=[min(df['time']), df['time'][60]],
        
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label="1wk",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1MO",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6MO",
                     step="month",
                     stepmode="backward"),
                dict(step="all"),
            ])
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '95vh'}
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=9001, debug=True)

Dash app running on http://127.0.0.1:9001/


/home/mcwaage1/venv/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



# Floors

In [59]:
floors_df = pd.read_csv('~/qs/fitbit/data/floors.csv')
floors_df

,Date,Number of Floors
0,2020-01-01,0
1,2020-01-02,0
2,2020-01-03,1
3,2020-01-04,8
4,2020-01-05,2
...,...,...
362,2020-12-28,3
363,2020-12-29,1
364,2020-12-30,2
365,2020-12-31,1


In [60]:
# https://plotly.com/python/dropdowns/#relayout-dropdown
# https://dash.plotly.com/dash-core-components/dropdown
# add dropdown to update traces between meters and floors


df = floors_df

title = 'Fitbit Floors Dashboard'

mean_elevation_floors = round(df['Number of Floors'].mean(), 2)
max_elevation_floors = round(df['Number of Floors'].max(), 2)
min_elevation_floors = round(df['Number of Floors'].min(), 2)

rolling=7
rolling_elevation_floors = round(df['Number of Floors'].rolling(rolling).mean(), 2)


fig = go.Figure()

fig.add_trace(go.Bar(x=df['Date'], y=df['Number of Floors'], name='Number of Floors'))

fig.add_trace(go.Scatter(x=df['Date'], y=rolling_elevation_floors, name=f'{rolling} Day Rolling Number of Floors'))

fig.add_hline(y=mean_elevation_floors, line_dash="dash")

fig.update_yaxes(title_text="Floors")
fig.update_xaxes(title_text="Date")
fig.update_layout(title=dict(text=title, 
                             yanchor="top",
                             y=.95,
                             xanchor="left",
                             x=.05),
                 legend=dict(orientation="h",
                             yanchor="bottom",
                             y=1.02,
                             xanchor="right",
                             x=1))

fig.update_layout(
    hovermode='x unified',
    dragmode='pan',
    annotations=[
        #https://plotly.com/python/text-and-annotations/#adding-annotations-with-xref-and-yref-as-paper
        #https://community.plotly.com/t/how-to-add-a-text-area-with-custom-text-as-a-kind-of-legend-in-a-plotly-plot/24349/3
        go.layout.Annotation(
            text=f'Max Floors: {max_elevation_floors}<br>Min Floors: {min_elevation_floors}<br>Mean Floors: {mean_elevation_floors}',
            align='left',
            showarrow=False,
            xref='paper',
            yref='paper',
            yanchor="top",
            y=.98,
            xanchor="right",
            x=1,
          )],
    xaxis=dict(
        
        # change to set view window
        # range=[min(df['time']), df['time'][60]],
        
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label="1wk",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1MO",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6MO",
                     step="month",
                     stepmode="backward"),
                dict(step="all"),
            ])
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '95vh'}
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=9020, debug=True)

Dash app running on http://127.0.0.1:9020/


# Sleep

In [182]:
sleep_df = pd.read_csv('~/qs/fitbit/data/sleep.csv')
sleep_df

,Start,End,Day
0,1970-01-01 23:00:00,1970-01-01 23:59:59,2019-12-31
1,1970-01-01 00:00:01,1970-01-01 03:58:00,2020-01-01
2,1970-01-01 20:40:00,1970-01-01 23:59:59,2020-01-01
3,1970-01-01 00:00:01,1970-01-01 07:12:30,2020-01-02
4,1970-01-01 15:47:00,1970-01-01 23:59:59,2020-01-02
...,...,...,...
466,1970-01-01 10:51:00,1970-01-01 18:29:30,2020-12-28
467,1970-01-01 16:19:30,1970-01-01 23:59:59,2020-12-29
468,1970-01-01 00:00:01,1970-01-01 02:30:30,2020-12-30
469,1970-01-01 09:24:00,1970-01-01 18:09:00,2020-12-31


In [183]:
sleep_df['Start'] = pd.to_datetime(sleep_df['Start'])
sleep_df['End'] = pd.to_datetime(sleep_df['End'])

sleep_df

,Start,End,Day
0,1970-01-01 23:00:00,1970-01-01 23:59:59,2019-12-31
1,1970-01-01 00:00:01,1970-01-01 03:58:00,2020-01-01
2,1970-01-01 20:40:00,1970-01-01 23:59:59,2020-01-01
3,1970-01-01 00:00:01,1970-01-01 07:12:30,2020-01-02
4,1970-01-01 15:47:00,1970-01-01 23:59:59,2020-01-02
...,...,...,...
466,1970-01-01 10:51:00,1970-01-01 18:29:30,2020-12-28
467,1970-01-01 16:19:30,1970-01-01 23:59:59,2020-12-29
468,1970-01-01 00:00:01,1970-01-01 02:30:30,2020-12-30
469,1970-01-01 09:24:00,1970-01-01 18:09:00,2020-12-31


In [184]:
# https://stackoverflow.com/questions/52007139/get-the-average-date-from-multiple-dates-pandas
average_sleep_df = pd.DataFrame()
average_sleep_df['Start'] = [x.value for x in sleep_df['Start']]
average_sleep_df['End'] = [x.value for x in sleep_df['End']]

average_sleep_df.describe().apply(pd.to_datetime)


,Start,End
count,4.710000e+02,4.710000e+02
mean,3.500271e+13,5.780404e+13
std,2.392095e+13,2.283956e+13
min,1.000000e+09,1.050000e+12
25%,1.059000e+13,4.213500e+13
50%,3.900000e+13,6.399000e+13
75%,4.594500e+13,7.519500e+13
max,8.634000e+13,8.639900e+13


In [185]:
len(average_sleep_df)

471

In [186]:
# # https://stackoverflow.com/questions/52007139/get-the-average-date-from-multiple-dates-pandas
# average_sleep_df = pd.DataFrame()
# average_sleep_df['Start'] = [x.value for x in sleep_df['Start'] if x.value != 1000000000] # 1 second after midnight
# average_sleep_df['End'] = [x.value for x in sleep_df['End'] if x.value != 86399000000000] # 1 second before midnight

# average_sleep_df.describe().apply(pd.to_datetime)

In [187]:
len(average_sleep_df)

471

In [188]:
average_sleep_df

,Start,End
0,82800000000000,86399000000000
1,1000000000,14280000000000
2,74400000000000,86399000000000
3,1000000000,25950000000000
4,56820000000000,86399000000000
...,...,...
466,39060000000000,66570000000000
467,58770000000000,86399000000000
468,1000000000,9030000000000
469,33840000000000,65340000000000


In [189]:
# https://stackoverflow.com/questions/19231871/convert-unix-time-to-readable-date-in-pandas-dataframe
mean_sleep_start = pd.to_datetime(average_sleep_df['Start'].mean())
mean_sleep_end = pd.to_datetime(average_sleep_df['End'].mean())

In [190]:
mean_sleep_start

Timestamp('1970-01-01 09:43:22.713375796')

In [191]:
median_sleep_start = pd.to_datetime(average_sleep_df['Start'].median())
median_sleep_end = pd.to_datetime(average_sleep_df['End'].median())

In [192]:
median_sleep_start

Timestamp('1970-01-01 10:50:00')

In [193]:
std_sleep_start = pd.to_datetime(average_sleep_df['Start'].std())
std_sleep_end = pd.to_datetime(average_sleep_df['End'].std())

In [194]:
std_sleep_start

Timestamp('1970-01-01 06:38:40.954626324')

In [195]:
rolling=7
rolling_timestamp = average_sleep_df.rolling(rolling).mean()
rolling_timestamp['Start'] = rolling_timestamp['Start'].apply(pd.to_datetime)
rolling_timestamp['End'] = rolling_timestamp['End'].apply(pd.to_datetime)

rolling_timestamp.head(10)

,Start,End
0,NaT,NaT
1,NaT,NaT
2,NaT,NaT
3,NaT,NaT
4,NaT,NaT
5,NaT,NaT
6,1970-01-01 10:20:17.571428571,1970-01-01 15:27:03.857142857
7,1970-01-01 08:26:04.714285714,1970-01-01 13:59:51.142857142
8,1970-01-01 09:51:51.714285714,1970-01-01 15:29:25.428571428
9,1970-01-01 07:08:17.428571428,1970-01-01 13:20:12.714285714


In [196]:
sleep_df['7 Day Rolling Start Time'] = rolling_timestamp['Start']
sleep_df['7 Day Rolling End Time'] = rolling_timestamp['End']
sleep_df

,Start,End,Day,7 Day Rolling Start Time,7 Day Rolling End Time
0,1970-01-01 23:00:00,1970-01-01 23:59:59,2019-12-31,NaT,NaT
1,1970-01-01 00:00:01,1970-01-01 03:58:00,2020-01-01,NaT,NaT
2,1970-01-01 20:40:00,1970-01-01 23:59:59,2020-01-01,NaT,NaT
3,1970-01-01 00:00:01,1970-01-01 07:12:30,2020-01-02,NaT,NaT
4,1970-01-01 15:47:00,1970-01-01 23:59:59,2020-01-02,NaT,NaT
...,...,...,...,...,...
466,1970-01-01 10:51:00,1970-01-01 18:29:30,2020-12-28,1970-01-01 09:37:51.714285714,1970-01-01 15:10:17.000000000
467,1970-01-01 16:19:30,1970-01-01 23:59:59,2020-12-29,1970-01-01 11:57:47.285714285,1970-01-01 18:33:29.714285714
468,1970-01-01 00:00:01,1970-01-01 02:30:30,2020-12-30,1970-01-01 10:19:30.285714285,1970-01-01 16:20:38.285714285
469,1970-01-01 09:24:00,1970-01-01 18:09:00,2020-12-31,1970-01-01 08:42:00.285714285,1970-01-01 15:30:29.857142857


In [197]:
customdata = ['7 Day Rolling Start Time', '7 Day Rolling End Time']

In [198]:
# https://community.plotly.com/t/how-do-i-change-the-x-axis-representation-of-time-on-the-px-timeline/44834/4
# https://stackoverflow.com/questions/67319505/how-to-plot-multiple-timelines-in-one-graph
# https://stackoverflow.com/questions/66905297/plotly-is-it-possible-to-add-a-second-y-axis-to-plotly-timeline

df = sleep_df

title = 'Sleep Timeline Dashboard'

fig = go.Figure()
fig = px.timeline(df, x_start="Start", x_end="End", y='Day')

# # start/end marker traces
# # https://plotly.com/python/marker-style/
# fig.add_trace(go.Scatter(x=df['Start'], y=df['Day'], name='Starts', mode='markers', marker_color='#00CC96'))
# fig.add_trace(go.Scatter(x=df['End'], y=df['Day'], name='Ends', mode='markers', marker_color='#EF553B'))

# #rolling
# fig.add_trace(go.Scatter(x=df['7 Day Rolling Start Time'], 
#                          y=df['Day'], 
#                          name=f'{rolling} Day Rolling Start Time',
#                          customdata = customdata, 
#                          hovertemplate =
#                          "{rolling} Day Rolling Mean Start Time: %{x}<br>" +
#                          "<extra></extra>",
#                         ))
# fig.add_trace(go.Scatter(x=df['7 Day Rolling End Time'], 
#                          y=df['Day'], 
#                          name=f'{rolling} Day Rolling End Time',
#                          customdata = customdata, 
#                          hovertemplate =
#                          "{rolling} Day Rolling Mean End Time: %{x}<br>" +
#                          "<extra></extra>",
#                         ))

fig.add_vline(x=median_sleep_start, line_dash="dash", opacity=0.5)
fig.add_vline(x=median_sleep_end, line_dash="dash", opacity=0.5)

# https://github.com/d3/d3-time-format#locale_format change time format
fig.update_layout(xaxis=dict(
                      title='Timestamp', 
                      tickformat = '%H:%M:%S',
                      range=['1970-01-01 00:00:00','1970-01-01 23:59:59']
                  ),
                  yaxis=dict(
                      title='Day',
                      tickformat = '%b/%d/%Y',
                      #dtick=86400000*2
                   ),
                 showlegend=True)
    
fig.update_layout(#hovermode='x unified',
                  #dragmode='pan',
                  title=dict(text=title, 
                             yanchor="top",
                             y=.98,
                             xanchor="left",
                             x=.05),
                 legend=dict(orientation="h",
                             yanchor="bottom",
                             y=1.02,
                             xanchor="right",
                             x=1,
                             ))

fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '98vh',
               #'width': '150vh',
              }
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=9010, debug=True)

Dash app running on http://127.0.0.1:9010/


/home/mcwaage1/venv/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



# Sleep Start/End Times Histogram

In [199]:
# need to fix start/end times

title = "Sleep Start/End Times Histogram"

df = sleep_df

fig = go.Figure()
fig.add_trace(go.Histogram(x=df['Start'], name="Starts"))
fig.add_trace(go.Histogram(x=df['End'], name='Ends'))

# Overlay both histograms
fig.update_layout(barmode='overlay')

# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75, 
                  # https://plotly.com/python/reference/histogram/#histogram-xbins
                  xbins_size=60000*10,
                  #xbins_start=60000,
                  #xbins_end=86394000,
                 )

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '98vh',
               #'width': '150vh',
              }
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=9020, debug=True)['sleeping_durations_seconds']

Dash app running on http://127.0.0.1:9020/


# Sleep Duration Histogram

In [200]:
df['Activity'] = 'Sleeping'
df['Duration in Seconds'] = pd.to_timedelta(df['End'] - df['Start']).astype('timedelta64[s]')

df

,Start,End,Day,7 Day Rolling Start Time,7 Day Rolling End Time,Activity,Duration in Seconds
0,1970-01-01 23:00:00,1970-01-01 23:59:59,2019-12-31,NaT,NaT,Sleeping,3599.0
1,1970-01-01 00:00:01,1970-01-01 03:58:00,2020-01-01,NaT,NaT,Sleeping,14279.0
2,1970-01-01 20:40:00,1970-01-01 23:59:59,2020-01-01,NaT,NaT,Sleeping,11999.0
3,1970-01-01 00:00:01,1970-01-01 07:12:30,2020-01-02,NaT,NaT,Sleeping,25949.0
4,1970-01-01 15:47:00,1970-01-01 23:59:59,2020-01-02,NaT,NaT,Sleeping,29579.0
...,...,...,...,...,...,...,...
466,1970-01-01 10:51:00,1970-01-01 18:29:30,2020-12-28,1970-01-01 09:37:51.714285714,1970-01-01 15:10:17.000000000,Sleeping,27510.0
467,1970-01-01 16:19:30,1970-01-01 23:59:59,2020-12-29,1970-01-01 11:57:47.285714285,1970-01-01 18:33:29.714285714,Sleeping,27629.0
468,1970-01-01 00:00:01,1970-01-01 02:30:30,2020-12-30,1970-01-01 10:19:30.285714285,1970-01-01 16:20:38.285714285,Sleeping,9029.0
469,1970-01-01 09:24:00,1970-01-01 18:09:00,2020-12-31,1970-01-01 08:42:00.285714285,1970-01-01 15:30:29.857142857,Sleeping,31500.0


In [201]:
unique_activities_sorted = ['Sleeping']

In [202]:
merged_df_list = []
for activity in unique_activities_sorted:
    globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame()
    globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day'] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Day']
    globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')] =  df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']       
    globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day'] =  [pd.to_datetime(i) for i in globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day']]
    globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')] = [int(i) for i in globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')]]
    
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame(df.loc[df['Activity'].str.contains(f"{activity}")].groupby(df['Day']).agg('Duration in Seconds').sum()).reset_index().rename({'Duration in Seconds':f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')}, axis='columns')
    globals()[f"grouped_{activity}_count_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame(df.loc[df['Activity'].str.contains(f"{activity}")].groupby(df['Day']).agg('Duration in Seconds').count()).reset_index().rename({'Duration in Seconds':f"{activity}_counts".lower().replace('/', '_').replace(' ', '_')}, axis='columns')
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_counts".lower().replace('/', '_').replace(' ', '_')] = globals()[f"grouped_{activity}_count_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_counts".lower().replace('/', '_').replace(' ', '_')]
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day'] =  [pd.to_datetime(i) for i in globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day']]
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')] = [int(i) for i in globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')]]

    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].set_index('Day')
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].asfreq('D', fill_value=0).reset_index()
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = 0
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds_7_day_rolling_mean".lower().replace('/', '_').replace(' ', '_')] = 0

    for i in globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')].index:
        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations".lower().replace('/', '_').replace(' ', '_')][i] = str(pd.Timedelta(seconds=globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')][i]))[-8:]
        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds_7_day_rolling_mean".lower().replace('/', '_').replace(' ', '_')][i] = np.nan_to_num(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')].rolling(7).mean()[i])

    merged_df_list.append(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')])
    
grouped_sleeping_df

<ipython-input-202-feae6b683172>:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/mcwaage1/venv/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-202-feae6b683172>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Day,sleeping_durations_seconds,sleeping_counts,sleeping_durations,sleeping_durations_seconds_7_day_rolling_mean
0,2019-12-31,3599,1,00:59:59,0
1,2020-01-01,26278,2,07:17:58,0
2,2020-01-02,55528,2,15:25:28,0
3,2020-01-03,43439,2,12:03:59,0
4,2020-01-04,14940,1,04:09:00,0
...,...,...,...,...,...
363,2020-12-28,27510,1,07:38:30,30235
364,2020-12-29,27629,1,07:40:29,28833
365,2020-12-30,9029,1,02:30:29,25182
366,2020-12-31,31500,1,08:45:00,26168


In [207]:
df = grouped_sleeping_df
df_counts = [i for i in df.columns if i.endswith('counts')]
df_durations_seconds = [i for i in df.columns if i.endswith('seconds')]
df_durations = [i for i in df.columns if i.endswith('durations')]
df_durations_seconds_7_day_rolling_mean = [i for i in df.columns if i.endswith('mean')]
df_durations

['sleeping_durations']

In [208]:
customdata = ['sleeping_durations']

In [209]:
# sleep duration histogram
title = 'Sleep Duration Histogram'

df = grouped_sleeping_df
customdata=[df[col] for col in df[df_durations].columns],

rolling = 7

fig = px.histogram(df, 
                   #x=df['Day'], 
                   x=[df[col] for col in df[df_durations_seconds].columns],
#                    color_discrete_map=correlation_colors,
                   title=title,
                   nbins=60,
                  )

fig.update_traces(customdata=customdata,
                  hovertemplate =
                  "Activity: Sleeping<br>" +
                  "Counts: %{y}<br>" +
                  "Duration in Seconds: %{x}<br>" +
#                   "Duration: %{customdata}<br>" +
                  "<extra></extra>",
                  #width=10,
                 )

# https://stackoverflow.com/questions/61406968/change-colors-in-python-dash-plotly-theme
fig.update_layout(#barmode='stack',
                  #lot_bgcolor='#282F44',
                  #paper_bgcolor ='#282F44',
                  #font_color='#FFF',
                  #modebar_add="v1hovermode",#+hoverclosest+hovercompare+togglehover+togglespikelines",
                  #hovermode='x unified',
                  
                  # https://github.com/d3/d3-time-format#locale_format change time format
                  xaxis=dict(title='Sleep Durations with ',
                            ),
                  yaxis=dict(title='Sleeping Counts Within Specified Bin',
#                              fixedrange=True, # for keeping zoom level to entire days
                            ),
                  #hovermode='x unified',
                  #dragmode='pan',
                  title=dict(text=title, 
                             yanchor="top",
                             y=.98,
                             xanchor="left",
                             x=.05),
                  legend=dict(title='Activities',
                              itemclick='toggle',
                              itemdoubleclick='toggleothers',
                              # for putting legend below chart 
#                               orientation="h",
#                               yanchor="bottom",
#                               y=-.1,
#                               xanchor="right",
#                               x=1,
                             ),
                 )

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")
              
app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '98vh',
               'width': '98vw',
              },
        
#         style={'height': '950px',
#                'width': '1600px',
#               },
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=9030, debug=True)

Dash app running on http://127.0.0.1:9030/


/home/mcwaage1/venv/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



# Sleeping Duration Scatterplot

In [217]:
grouped_sleeping_df.dtypes

Day                                              datetime64[ns]
sleeping_durations_seconds                                int64
sleeping_counts                                           int64
sleeping_durations                                       object
sleeping_durations_seconds_7_day_rolling_mean             int64
dtype: object

In [220]:
grouped_sleeping_df['sleeping_durations_seconds'] = pd.to_datetime(grouped_sleeping_df['sleeping_durations_seconds'], unit='s')
grouped_sleeping_df.dtypes

Day                                              datetime64[ns]
sleeping_durations_seconds                       datetime64[ns]
sleeping_counts                                           int64
sleeping_durations                                       object
sleeping_durations_seconds_7_day_rolling_mean             int64
dtype: object

In [222]:
title = 'Sleeping Timeline Scatter Plot'

df = grouped_sleeping_df
customdata=[df[col] for col in df[df_durations].columns],

rolling = 7

fig = px.scatter(df, 
                 x=df['Day'], 
                 y=[df[col] for col in df[df_durations_seconds].columns],
#                  color_discrete_map=correlation_colors,
                 title=title,
                 # https://plotly.com/python/linear-fits/
                 trendline="ols",
#                  trendline="rolling",
#                  trendline_options=dict(window=rolling),
                )

fig.update_traces(customdata=customdata,
                  hovertemplate =
                  "Activity: Sleeping<br>" +
                  "Day: %{x}<br>" +
                  "Duration in Seconds: %{y}<br>" +
                  "Duration: %{customdata}<br>" +
                  "<extra></extra>",
                  #width=10,
                 ),

# https://stackoverflow.com/questions/61406968/change-colors-in-python-dash-plotly-theme
# https://github.com/d3/d3-time-format#locale_format change time format
fig.update_layout(xaxis=dict(title='Day',
                            ),
                  yaxis=dict(title='Sleeping Durations',
                             tickformat = '%H:%M:%S',
                             range=['1970-01-01 00:00:00','1970-01-01 23:59:59'],
                            ),
                  title=dict(text=title, 
                             yanchor="top",
                             y=.98,
                             xanchor="left",
                             x=.05),
                  legend=dict(title='Activities',
                              itemclick='toggle',
                              itemdoubleclick='toggleothers',
                             ),
                    )

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")
              
app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '98vh',
               'width': '98vw',
              },
        
#         style={'height': '950px',
#                'width': '1600px',
#               },
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=8090, debug=True)

UFuncTypeError: ufunc 'subtract' cannot use operands with types dtype('<M8[ns]') and dtype('O')

In [165]:
# Example of another histogram

# title = f"Number of Days <br>At Specified {series.name} Level"

# fig = go.Figure(data=[go.Histogram(x=series)])

# fig.update_layout(title=dict(text=title,
#                              yanchor="top",
#                              y=.85,
#                              xanchor="left",
#                              x=.075),
#                   legend=dict(orientation="h",
#                               yanchor="bottom",
#                               y=1.02,
#                               xanchor="right",
#                               x=1),
#                  bargap=0.1)
# fig.update_traces(xbins_start=min(series), xbins_size=bins, selector=dict(type='histogram'))
# fig.update_xaxes(tickmode='linear', dtick=bins, tickangle=tickangle)
# fig.update_xaxes(title_text=f"{series.name} Level")
# fig.update_yaxes(title_text=title)
# html.Div([
#     dcc.Graph(
#         figure=fig
#     )
# ])

In [84]:
#https://plotly.com/python/discrete-color/
print(px.colors.qualitative.Plotly)

['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']


In [90]:
fig = px.colors.qualitative.swatches()
fig.show()